<a href="https://colab.research.google.com/github/clarari137/Home_Sales/blob/main/Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.2'
spark_version = 'spark-3.4.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"),sep=',', header=True)
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""select round(avg(price),2), date_built, count(*) from home_sales where bedrooms='4' group by date_built""").show()


+--------------------+----------+--------+
|round(avg(price), 2)|date_built|count(1)|
+--------------------+----------+--------+
|           296050.24|      2016|    1336|
|           298233.42|      2012|    1366|
|           296576.69|      2017|    1380|
|           299073.89|      2014|    1314|
|           299999.39|      2013|    1342|
|            302141.9|      2011|    1315|
|           307908.86|      2015|    1445|
|           296800.75|      2010|    1340|
+--------------------+----------+--------+



In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""select round(avg(price),2), date_built, count(*) from home_sales where bedrooms='3' and bathrooms='3' group by date_built""").show()


+--------------------+----------+--------+
|round(avg(price), 2)|date_built|count(1)|
+--------------------+----------+--------+
|           290555.07|      2016|     676|
|           293683.19|      2012|     641|
|           292676.79|      2017|     677|
|           290852.27|      2014|     695|
|           295962.27|      2013|     641|
|           291117.47|      2011|     668|
|            288770.3|      2015|     708|
|           292859.62|      2010|     690|
+--------------------+----------+--------+



In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""select round(avg(price),2), date_built, count(*)
              from home_sales
              where bedrooms='3' and bathrooms='3' and floors='2' and sqft_living>'2,000'
              group by date_built""").show()


+--------------------+----------+--------+
|round(avg(price), 2)|date_built|count(1)|
+--------------------+----------+--------+
|            293965.1|      2016|     172|
|           307539.97|      2012|     154|
|           280317.58|      2017|     182|
|           298264.72|      2014|     174|
|           303676.79|      2013|     160|
|           276553.81|      2011|     171|
|           297609.97|      2015|     187|
|           285010.22|      2010|     176|
+--------------------+----------+--------+



In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
spark.sql("""select round(avg(price),2), view
              from home_sales
              group by view
              having round(avg(price),2)>=350000""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+----+
|round(avg(price), 2)|view|
+--------------------+----+
|           788128.21|  51|
|           798684.82|  54|
|           750537.94|  69|
|           1072285.2|  87|
|           752861.18|  73|
|           767036.67|  64|
|            791453.0|  59|
|          1056336.74|  85|
|           733780.26|  52|
|            775651.1|  71|
|          1053739.33|  98|
|          1061201.42|  99|
|          1017815.92|  96|
|           1026669.5| 100|
|           695865.58|  70|
|           746877.59|  61|
|          1114042.94|  75|
|          1080649.37|  78|
|          1107839.15|  89|
|          1076205.56|  77|
+--------------------+----+
only showing top 20 rows

--- 0.4187610149383545 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""select round(avg(price),2), view
              from home_sales
              group by view
              having round(avg(price),2)>=350000""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+--------------------+----+
|round(avg(price), 2)|view|
+--------------------+----+
|           788128.21|  51|
|           798684.82|  54|
|           750537.94|  69|
|           1072285.2|  87|
|           752861.18|  73|
|           767036.67|  64|
|            791453.0|  59|
|          1056336.74|  85|
|           733780.26|  52|
|            775651.1|  71|
|          1053739.33|  98|
|          1061201.42|  99|
|          1017815.92|  96|
|           1026669.5| 100|
|           695865.58|  70|
|           746877.59|  61|
|          1114042.94|  75|
|          1080649.37|  78|
|          1107839.15|  89|
|          1076205.56|  77|
+--------------------+----+
only showing top 20 rows

--- 0.27994728088378906 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").parquet('parquet_home_sales', mode='overwrite')

In [ ]:
# 11. Read the parquet formatted data.
p_df=spark.read.parquet('parquet_home_sales')

In [ ]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_home_sales')

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""select round(avg(price),2), view
              from p_home_sales
              group by view
              having round(avg(price),2)>=350000""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+----+
|round(avg(price), 2)|view|
+--------------------+----+
|           788128.21|  51|
|           798684.82|  54|
|           750537.94|  69|
|           752861.18|  73|
|           1072285.2|  87|
|           767036.67|  64|
|            791453.0|  59|
|          1056336.74|  85|
|           733780.26|  52|
|            775651.1|  71|
|          1053739.33|  98|
|          1061201.42|  99|
|          1017815.92|  96|
|           1026669.5| 100|
|           695865.58|  70|
|           746877.59|  61|
|          1114042.94|  75|
|          1080649.37|  78|
|          1107839.15|  89|
|          1076205.56|  77|
+--------------------+----+
only showing top 20 rows

--- 0.6985061168670654 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached("home_sales"):
  print("a table is till cached")
else:
  print("all clear")


all clear
